# How to build simple Agent LLM App with LangGraph

## Concepts included
* LangGraph

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

In [3]:
# !pip list | grep langchain
!pip list | findstr langchain

langchain                                0.3.13
langchain-chroma                         0.1.4
langchain-community                      0.3.13
langchain-core                           0.3.43
langchain-experimental                   0.3.4
langchain-google-community               2.0.7
langchain-groq                           0.2.5
langchain-openai                         0.2.14
langchain-text-splitters                 0.3.4
langchainhub                             0.1.21


## Connect with LLM and start conversation with it

In [4]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(model=MODEL_GPT)

### Track operation in LangSmith
* [Open LangSmith here](https://smith.langchain.com/)

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take

## Tool-using Agent
* We will need to add our Tavily API key in .env file
* TAVILY_API_KEY=

In [5]:
#pip install langgraph

### Define tools

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'url': 'https://www.nbcsports.com/soccer/news/euro-2024-player-power-rankings-who-are-the-top-20-stars',
  'content': '- NBC Sports All Sports All Sports Fantasy Sports Basketball Player News All Sports Fantasy Sports Basketball Player News With Kylian Mbappe, Harry Kane and Kevin de Bruyne aiming to lead their teams to glory, plus ageing superstars like Toni Kroos and Cristiano Ronaldo still going strong and young stars Jude Bellingham, Florian Wirtz and Lamine Yamal set to take over, you can look all over the place for star performances this summer. How to watch Aston Villa vs Tottenham Hotspur live: Stream link, TV channel, team news, prediction How to watch Plymouth vs Liverpool live: FA Cup stream link, TV channel, team news, prediction Sports Sports Podcasts Add favorite players, teams, and leagues with an NBCUniversal Profile'},
 {'url': 'https://bleacherreport.com/articles/10124474-ranking-the-top-10-players-at-euro-2024',
  'content': 'Ranking the Top 10 Players at Euro 2024

In [7]:
tools = [search]

In [8]:
# In order to enable this model to do tool calling we use .bind_tools to give language model knowledge of these tools

llm_with_tools = llm.bind_tools(tools)

## Create agent

In [9]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

## Run agent

In [10]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', additional_kwargs={}, response_metadata={}, id='a5a0d3cc-8f18-4dee-ac7d-73c22c1e7b61'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JKZo4O5GB16FSoGYh3I0JHDU', 'function': {'arguments': '{"query":"Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d0f04f76-3cb6-4590-b4c2-444b06f11efa-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Eurocup 2024 location'}, 'id': 'call_JKZo4O5GB16FSo

In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PtTe1IYQVwadQq1aVPiCUYkk', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 94, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9cc49b64-4905-46de-b78f-cb9471d51f08-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_PtTe1IYQVwadQq1aVPiCUYkk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 